Word list from [mit](http://www.mit.edu/~ecprice/wordlist.10000)

In [1]:
import collections
import random
import string
from IPython.display import clear_output
import time

In [2]:
with open("wordlist.10000.txt") as f:
    CORPUS = f.read().split()

In [3]:
Learnable = collections.namedtuple("Learnable", ["solved", "number_hits", "last_move"])

In [4]:
class Game:
    def __init__(self, solution, players=None):
        self.solution = solution
        self.players = players if players else [0]
        self.remaining_letters = [set(self.solution) for _ in self.players]
        self.solved = [False for _ in self.players]

    def render(self, player_index=0):
        rendering = ""
        for letter in self.solution:
            if letter in self.remaining_letters[player_index]:
                rendering = rendering + "_"
            else:
                rendering = rendering + letter
        return rendering

    def play_move(self, letter, player_index=0):
        self.solved[player_index] = self._play_move(letter, player_index)
        return Learnable(
            solved=self.solved[player_index],
            number_hits=self.solution.count(letter),
            last_move=letter,
        )

    def play_game(self):
        while not all(self.solved):
            for i, player in enumerate(self.players):
                if not self.solved[i]:
                    player.learn(self.play_move(player.move(), i))

    def _play_move(self, letter, player_index):
        self.remaining_letters[player_index].discard(letter)
        return len(self.remaining_letters[player_index]) == 0

In [5]:
class RandomGame(Game):
    def __init__(self, corpus, random_seed=None, players=None):
        if random_seed:
            random.seed(random_seed)
        self.corpus = corpus
        solution_ = self._random_word()
        super().__init__(solution=solution_, players=players)

    def _random_word(self):
        return random.choice(self.corpus)

In [6]:
class Player:
    def __init__(self, strategy=None, corpus=None):
        self.corpus = corpus
        self.corpus_left = corpus
        self.strategy = strategy
        self.guesses = []
        self.errors = []
        self.alpha_left = string.ascii_lowercase
        self.move_list_0 = self.strategy.init_moves(self.corpus)
        self.move_list = self.move_list_0.copy()

    def move(self):
        move_ = self.move_list.pop(0)
        self.guesses.append(move_)
        self.alpha_left = self.alpha_left.replace(move_, "")
        return move_

    def learn(self, learnable):
        if learnable.number_hits == 0:
            self.errors.append(learnable.last_move)
        self.corpus_left, new_moves = self.strategy.recalculate(
            learnable, self.corpus_left, self.alpha_left
        )
        if new_moves:
            self.move_list = new_moves

    def reset(self):
        self.alpha_left = string.ascii_lowercase
        self.num_moves = 0
        self.guessed = []
        self.corpus_left = self.corpus
        self.move_list = self.move_list_0.copy()
        self.errors = []

In [7]:
class Strategy:
    def __init__(
        self,
        random_move=None,  # first, all
        letter_frequency="char",  # word
        learn=None,  # hit, frequency
    ):
        self.random_move = random_move
        self._get_moves_freq = {
            "char": self._get_moves_freq_char,
            "word": self._get_moves_freq_word,
        }.get(letter_frequency, None)
        self.learn = learn

    def init_moves(self, corpus):
        if self.random_move == "first":
            move_list_a = [self._random_moves()[0]]
            move_list_b = self._get_moves_freq(corpus).remove(move_list_a[0])
            move_list = move_list_a + move_list_b
        elif self.random_move == "all":
            move_list = self._random_moves()
        else:
            move_list = self._get_moves_freq(corpus)
        return move_list

    def recalculate(self, learnable, corpus, alpha_left):
        if self.learn is None:
            return (corpus, None)
        new_corpus = self._prune_corpus(learnable, corpus)
        new_move_list = self._get_moves_freq(new_corpus)
        new_move_list = [move for move in new_move_list if move in alpha_left]
        return (new_corpus, new_move_list)

    def _random_moves(self):
        new_move_list = list(string.ascii_lowercase)
        random.shuffle(new_move_list)
        return new_move_list

    def _get_moves_freq_char(self, corpus):
        c = "".join(["".join(word for word in corpus)])
        return [elem[0] for elem in collections.Counter(c)]

    def _get_moves_freq_word(self, corpus):
        c = "".join(["".join("".join((set(word))) for word in corpus)])
        return [elem[0] for elem in collections.Counter(c)]

    def _prune_corpus(self, learnable, corpus):
        if self.learn == "hit":
            new_corpus = self._corpus_letter_check(learnable, corpus)
        elif self.learn == "frequency":
            new_corpus = self._corpus_frequency_check(learnable, corpus)
        return new_corpus

    def _corpus_letter_check(self, learnable, corpus):
        letter = learnable.last_move
        if learnable.number_hits > 0:
            new_corpus = [word for word in corpus if letter in word]
        else:
            new_corpus = [word for word in corpus if letter not in word]
        return new_corpus

    def _corpus_frequency_check(self, learnable, corpus):
        letter = learnable.last_move
        hits = learnable.number_hits
        if hits > 0:
            new_corpus = [word for word in corpus if word.count(letter) == hits]
        else:
            new_corpus = [word for word in corpus if letter not in word]
        return new_corpus

In [8]:
s = Strategy(letter_frequency="word")
t = Strategy(letter_frequency="word", learn="hit")
u = Strategy(letter_frequency="word", learn="frequency")

p = Player(strategy=s, corpus=CORPUS)
q = Player(strategy=t, corpus=CORPUS)
r = Player(strategy=u, corpus=CORPUS)

g = RandomGame(corpus=CORPUS, players=[p, q, r])
print(g.solution)
for player in [p, q, r]:
    player.reset()
g.play_game()
print(f"p: {len(p.guesses)} guesses; {len(p.errors)} errors")
print(f"q: {len(q.guesses)} guesses; {len(q.errors)} errors")
print(f"r: {len(r.guesses)} guesses; {len(r.errors)} errors")

prefer
p: 25 guesses; 21 errors
q: 20 guesses; 16 errors
r: 17 guesses; 13 errors


In [9]:
s = Strategy(letter_frequency="word")
t = Strategy(letter_frequency="word", learn="hit")
u = Strategy(letter_frequency="word", learn="frequency")

p = Player(strategy=s, corpus=CORPUS)
q = Player(strategy=t, corpus=CORPUS)
r = Player(strategy=u, corpus=CORPUS)

errors = [[], [], []]
moves = [[], [], []]

t0 = time.process_time()
for i_w, word in enumerate(CORPUS):
    g = Game(solution=word, players=[p, q, r])
    g.play_game()
    for i, player in enumerate(g.players):
        errors[i].append(len(player.errors))
        moves[i].append(len(player.guesses))
        player.reset()
td = time.process_time() - t0

print(f"Completed in {td} seconds.")

Completed in 599.507796 seconds.


Almost exactly 10 minutes.

In [10]:
wins = [0, 0, 0]
for i in range(len(g.players)):
    wins[i] = len([game for game in errors[i] if game < 6])

print(wins)

[852, 1346, 2777]
